In [19]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
from localcider.sequenceParameters import SequenceParameters
import scipy
from scipy.spatial.distance import cdist

In [19]:
path: str = './data/' #folder where files are stored
prokaryotes: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_unique_prot.csv"), dtype=str)   # imports file from the folder as species, based on name
prokaryotes_all: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_all.csv"), dtype=str) # imports 3rd file from the folder, based on name

In [17]:
import numpy as np
import os
import scipy
from scipy.spatial.distance import cdist
path_pdb = './data/pdbs'

Salt_bridges = ({})
Asp_Glu_array = np.empty((0, 4))
Lys_Arg_His_array = np.empty((0, 4))
with open(os.path.join(path_pdb, 'AF-C0H3V2-F1.pdb')) as f:
    for line in f:
        line = line.strip()
        if line.startswith('ATOM'):
            if ('ASP' in line and 'OD' in line) or ('GLU' in line and 'OE' in line):
                line_array = np.array([[line[7:12].strip(), line[27:38].strip(), line[39:46].strip(), line[47:54].strip()]])
                line_array = line_array.astype('float64')
                Asp_Glu_array = np.append(Asp_Glu_array, line_array, axis = 0)
            if ('LYS' in line and 'NZ' in line) or ('ARG' in line and 'NH' in line) or ('HIS' in line and 'NE' in line) or ('HIS' in line and 'ND' in line):
                line_array = np.array([[line[7:12].strip(), line[27:38].strip(), line[39:46].strip(), line[47:54].strip()]])
                line_array = line_array.astype('float64')
                Lys_Arg_His_array = np.append(Lys_Arg_His_array, line_array, axis = 0)

    #calculate distance clean up array
    distance = cdist(Asp_Glu_array[:,1:], Lys_Arg_His_array[:,1:], metric='euclidean') #calculate distance
    distance = np.concatenate((np.array([Lys_Arg_His_array[:,0]]), distance), axis=0) #add atom number from Lys_Arg_His to array
    distance = np.concatenate((np.insert(np.array([Asp_Glu_array[:,0]]), 0, None).reshape(-1,1), distance), axis=1) #add atom number from Asp_Glu to array
    distance[1:, 1:][distance[1:, 1:] > 4] = np.nan #set distance > 4 to nan
    rows_with_nan = np.insert(np.array([np.all(np.isnan(distance[1:, 1:]), axis=1)]),0, None) #find rows with all nan values
    cols_with_nan = np.insert(np.array([np.all(np.isnan(distance[1:, 1:]), axis=0)]),0, None) #find columns with all nan values
    distance = distance[~rows_with_nan, :] #delete rows with all nan values
    distance = distance[:, ~cols_with_nan] #delete columns with all nan values
    Salt_bridges['AF-C0H3V2-F1.pdb'] = distance

print(Asp_Glu_array)
print(Lys_Arg_His_array)
print(distance)

[[ 5.4000e+01 -2.0399e+01 -4.2400e-01  2.3480e+00]
 [ 5.5000e+01 -1.9003e+01  3.9100e-01  8.6600e-01]
 [ 1.4800e+02  5.7810e+00 -5.8270e+00 -9.9740e+00]
 [ 1.4900e+02  4.9100e+00 -7.2090e+00 -8.4830e+00]
 [ 1.5700e+02  8.6200e-01 -1.2699e+01 -5.2300e+00]
 [ 1.5800e+02  3.3900e-01 -1.1398e+01 -3.5520e+00]
 [ 2.3600e+02 -1.4320e+01 -7.4330e+00 -7.6370e+00]
 [ 2.3700e+02 -1.3646e+01 -7.5320e+00 -5.5380e+00]
 [ 2.8300e+02 -1.4901e+01 -8.3630e+00 -1.5474e+01]
 [ 2.8400e+02 -1.3392e+01 -8.1800e+00 -1.7070e+01]
 [ 2.9100e+02 -1.0042e+01 -4.1250e+00 -2.0281e+01]
 [ 2.9200e+02 -1.1279e+01 -2.5740e+00 -2.1247e+01]
 [ 3.5400e+02 -2.1310e+00 -1.8200e-01 -2.0955e+01]
 [ 3.5500e+02 -1.7710e+00  1.9170e+00 -2.0613e+01]
 [ 3.7400e+02  5.1670e+00  1.3590e+00 -8.8340e+00]
 [ 3.7500e+02  3.2430e+00  2.4000e+00 -9.0950e+00]
 [ 3.8300e+02  4.4790e+00 -3.0050e+00 -1.7163e+01]
 [ 3.8400e+02  4.5570e+00 -1.1870e+00 -1.8417e+01]
 [ 5.0900e+02 -4.8480e+00  1.2306e+01 -1.6123e+01]
 [ 5.1000e+02 -5.3080e+00  1.34

In [21]:
from function import salt_bridge
path_pdb = './data/pdbs'
Salt_Bridges = salt_bridge(path_pdb).keys()


In [ ]:
SeqObj = SequenceParameters('MADKDFGLNDIVEMKKPHPCGANSWKIIRMGMDIRIKCEGCSHSVMIPRREFERKLKKVLVKHEEPTS')
SeqObj.save_zscoresAndPlots(num_scrambles=100000, random_seed=None)


In [20]:
SecObj2 = SequenceParameters('MADKDFGLNDIVEMKKPHPCGANSWKIIRMGMDIRIKCEGCSHSVMIPRREFERKLKKVLVKHEEPTS')
SecObj2.get_kappa()

0.15316382219826957